In [13]:
import pandas as pd
import numpy as np

# Simulated data result

In [14]:
sim_result = pd.read_csv('result_sim_X_v2.csv')
sim_result.columns

Index(['mean_h_est', 'h_est_upper', 'h_est_lower', 'mean_var_genetic',
       'noise_var', 'global_pi', 'global_pi_upper', 'global_pi_lower',
       'global_pi_upper_1', 'global_pi_lower_1', 'FDR_50', 'sensitivity_50',
       'FDR_25', 'sensitivity_25', 'FDR_100', 'sensitivity_100', 'FDR_75',
       'mae_latent', 'mae_lasso', 'mae_ridge', 'n', 'p', 'linear_outcome',
       'mean_h_mcmc', 'low_h_mcmc', 'up_h_mcmc', 'global_pi_mcmc',
       'low_global_pi_mcmc', 'up_global_pi_mcmc', 'total_time_VI',
       'total_time_MCMC', 'FDR_100_mcmc', 'sensitivity_100_mcmc',
       'FDR_75_mcmc', 'sensitivity_75_mcmc', 'FDR_50_mcmc',
       'sensitivity_50_mcmc', 'FDR_25_mcmc', 'sensitivity_25_mcmc', 'true_h',
       'true_pi', 'beta_var', 'rho'],
      dtype='object')

In [15]:
sim_result = pd.read_csv('result_sim_X_v2.csv')
sim_result['h_cover'] = np.where(np.logical_and(sim_result.h_est_upper >sim_result.true_h, 
                                                sim_result.h_est_lower <sim_result.true_h), 1, 0)
sim_result['pi_cover'] = np.where(np.logical_and(sim_result.global_pi_upper >sim_result.true_pi, 
                                                sim_result.global_pi_lower <sim_result.true_pi), 1, 0)
sim_result['h_cover_mcmc'] = np.where(np.logical_and(sim_result.up_h_mcmc >sim_result.true_h, 
                                                sim_result.low_h_mcmc <sim_result.true_h), 1, 0)
sim_result['pi_cover_mcmc'] = np.where(np.logical_and(sim_result.up_global_pi_mcmc >sim_result.true_pi, 
                                                sim_result.low_global_pi_mcmc <sim_result.true_pi), 1, 0)
sim_result['FVE CI range'] = sim_result['h_est_upper'] - sim_result['h_est_lower']
sim_result['Percent Causal CI range'] = sim_result['global_pi_upper'] - sim_result['global_pi_lower']
sim_result.rename(columns = {'true_h':'True FVE',
                             'true_pi':'True Percent Causal',
                             'mean_h_est':'FVE VI',
                             'global_pi':'Percent Causal',
                             'h_cover':'Coverage FVE',
                             'pi_cover':'Coverage Percent Causal',
                             'mae_latent':'MAE VI',
                             'mae_lasso':'MAE LASSO',
                             'mae_ridge':'MAE Ridge',
                             }, 
                  inplace = True)
t = sim_result.\
    groupby(['linear_outcome','True FVE', 'True Percent Causal','n','p','rho']).\
    mean()[[ 'FVE VI','Coverage FVE','FVE CI range', 
            'Percent Causal', 'Coverage Percent Causal', 'Percent Causal CI range',
            "mean_h_mcmc", "global_pi_mcmc", "h_cover_mcmc", "pi_cover_mcmc","FDR_100_mcmc",
            "FDR_100"
            ]]
t

FVE VI  \
linear_outcome True FVE True Percent Causal n    p   rho             
False          0.8      0.1                 160  400 0    0.597633   
                                            320  400 0    0.756354   
                        0.3                 160  400 0    0.469353   
True           0.8      0.1                 160  400 0    0.645254   
                                            320  400 0    0.777637   
                                            800  400 0    0.796295   
                                            1600 400 0    0.798048   
                                            3200 400 0    0.799854   
                                            4800 400 0    0.799430   
                                            7200 400 0    0.799595   
                        0.3                 160  400 0    0.280491   
                                            320  400 0    0.653571   
                                            800  400 0    0.789472   
                                            1600 400 0    0.798107   
                                            3200 400 0    0.798135   
                                            4800 400 0    0.798748   
                                            7200 400 0    0.799586   

                                                          Coverage FVE  \
linear_outcome True FVE True Percent Causal n    p   rho                 
False          0.8      0.1                 160  400 0           0.100   
                                            320  400 0           0.855   
                        0.3                 160  400 0           0.000   
True           0.8      0.1                 160  400 0           0.180   
                                            320  400 0           0.700   
                                            800  400 0           0.925   
                                            1600 400 0           0.950   
                                            3200 400 0           0.925   
                                            4800 400 0           0.940   
                                            7200 400 0           0.910   
                        0.3                 160  400 0           0.000   
                                            320  400 0           0.005   
                                            800  400 0           0.770   
                                            1600 400 0           0.910   
                                            3200 400 0           0.945   
                                            4800 400 0           0.920   
                                            7200 400 0           0.940   

                                                          FVE CI range  \
linear_outcome True FVE True Percent Causal n    p   rho                 
False          0.8      0.1                 160  400 0        0.293103   
                                            320  400 0        0.147006   
                        0.3                 160  400 0        0.337855   
True           0.8      0.1                 160  400 0        0.155449   
                                            320  400 0        0.069631   
                                            800  400 0        0.039713   
                                            1600 400 0        0.027739   
                                            3200 400 0        0.019442   
                                            4800 400 0        0.015869   
                                            7200 400 0        0.012946   
                        0.3                 160  400 0        0.252592   
                                            320  400 0        0.111301   
                                            800  400 0        0.041832   
                                            1600 400 0        0.027979   
                                            3200 400 0        0.019704   
                                            4800 400 0        0.

# Main simulation to evaluate algorithm validality

In [16]:
sim_result = pd.read_csv('../simulation_result/result.csv')
chun_result = pd.read_csv('../simulation_result/chun_results.csv')
sim_result = pd.concat([sim_result,chun_result], axis = 1)

FileNotFoundError: [Errno 2] No such file or directory: '../simulation_result/result.csv'

In [ ]:
sim_result

,mean_h_est,h_est_upper,h_est_lower,mean_var_genetic,noise_var,global_pi,global_pi_upper,global_pi_lower,FDR,sensitivity,mae_latent,mae_lasso,mae_ridge,true_h,true_pi,beta_var,h2,mae
0,0.785826,0.828413,0.734330,11.958570,3.267713,0.076733,0.150797,0.025402,0.151515,0.318182,0.725920,1.014471,1.032728,0.80,0.100,0.1,0.433485,4.229898
1,0.781825,0.826767,0.730461,4.963962,1.390744,0.074199,0.144028,0.025030,0.218750,0.284091,0.492906,0.667637,0.657780,0.80,0.100,0.1,0.491880,1.471430
2,0.789928,0.828357,0.746261,8.731348,2.324597,0.076745,0.143884,0.029282,0.181818,0.306818,0.631336,0.859025,0.868210,0.80,0.100,0.1,0.390950,1.589764
3,0.786165,0.831914,0.734533,13.833671,3.773741,0.063757,0.136126,0.017645,0.071429,0.295455,0.842101,1.125438,1.124868,0.80,0.100,0.1,0.416797,2.780782
4,0.787803,0.829373,0.742744,9.420783,2.543586,0.078986,0.150605,0.027933,0.029412,0.375000,0.626751,0.899535,0.895874,0.80,0.100,0.1,0.463655,1.794607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.240832,0.307432,0.184613,0.585936,1.879982,0.010955,0.042728,0.000132,0.000000,1.000000,0.562440,0.637244,0.656467,0.25,0.005,0.1,0.101536,0.662705
5996,0.245624,0.311102,0.188273,0.160591,0.501665,0.011332,0.043627,0.000175,0.200000,1.000000,0.287781,0.338897,0.342997,0.25,0.005,0.1,0.147048,0.337225
5997,0.258520,0.325202,0.196579,0.192240,0.562163,0.009019,0.040227,0.000045,0.333333,0.500000,0.284216,0.335727,0.345284,0.25,0.005,0.1,0.149356,0.343391
5998,0.247079,0.331149,0.175694,0.631538,1.977193,0.013501,0.054969,0.000109,0.200000,1.000000,0.536385,0.626299,0.631932,0.25,0.005,0.1,0.095010,0.639185


In [ ]:
sim_result = pd.read_csv('../simulation_result/result.csv')
chun_result = pd.read_csv('../simulation_result/chun_results.csv')
sim_result = pd.concat([sim_result,chun_result], axis = 1)
sim_result['h_cover'] = np.where(np.logical_and(sim_result.h_est_upper >sim_result.true_h, 
                                                sim_result.h_est_lower <sim_result.true_h), 1, 0)
sim_result['pi_cover'] = np.where(np.logical_and(sim_result.global_pi_upper >sim_result.true_pi, 
                                                sim_result.global_pi_lower <sim_result.true_pi), 1, 0)
sim_result['FVE CI range'] = sim_result['h_est_upper'] - sim_result['h_est_lower']
sim_result['Percent Causal CI range'] = sim_result['global_pi_upper'] - sim_result['global_pi_lower']
sim_result.rename(columns = {'true_h':'True FVE',
                             'true_pi':'True Percent Causal',
                             'h2':'FVE GCTA',
                             'mean_h_est':'FVE VI',
                             'global_pi':'Percent Causal',
                             'h_cover':'Coverage FVE',
                             'pi_cover':'Coverage Percent Causal',
                             'mae_latent':'MAE VI',
                             'mae':'MAE GCTA', 
                             'mae_lasso':'MAE LASSO',
                             'mae_ridge':'MAE Ridge',
                             }, 
                  inplace = True)
t = sim_result.\
    groupby(['True FVE', 'True Percent Causal']).\
    mean()[['FVE GCTA', 'FVE VI','Coverage FVE','FVE CI range', 
            'Percent Causal', 'Coverage Percent Causal', 'Percent Causal CI range',
            'MAE GCTA','MAE VI', 'MAE LASSO', 'MAE Ridge']]

In [ ]:
print(
    t.transpose().round(3).to_latex()
)

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
True FVE & \multicolumn{4}{l}{0.25} & \multicolumn{4}{l}{0.50} & \multicolumn{4}{l}{0.80} \\
True Percent Causal &  0.005 &  0.010 &  0.050 &  0.100 &  0.005 &  0.010 &  0.050 &  0.100 &  0.005 &  0.010 &  0.050 &  0.100 \\
\midrule
FVE GCTA                &  0.124 &  0.127 &  0.129 &  0.131 &  0.260 &  0.263 &  0.265 &  0.269 &  0.438 &  0.447 &  0.449 &  0.450 \\
FVE VI                  &  0.241 &  0.241 &  0.236 &  0.230 &  0.487 &  0.487 &  0.479 &  0.472 &  0.792 &  0.791 &  0.786 &  0.780 \\
Coverage FVE            &  0.990 &  0.998 &  1.000 &  0.992 &  0.990 &  0.998 &  0.996 &  1.000 &  0.924 &  0.972 &  0.996 &  0.998 \\
FVE CI range            &  0.129 &  0.136 &  0.133 &  0.122 &  0.138 &  0.161 &  0.173 &  0.167 &  0.050 &  0.065 &  0.095 &  0.096 \\
Percent Causal          &  0.011 &  0.016 &  0.040 &  0.053 &  0.010 &  0.015 &  0.044 &  0.065 &  0.008 &  0.014 &  0.049 &  0.078 \\
Coverage Percent Causal &  1.000 &  1.000 &  1.000 &

/var/folders/cy/7rpv9wdx3nb25gzbj78xgqwm0000gn/T/ipykernel_2680/179635416.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  t.transpose().round(3).to_latex()


In [ ]:
sim_result

,mean_h_est,h_est_upper,h_est_lower,mean_var_genetic,noise_var,global_pi,global_pi_upper,global_pi_lower,FDR,sensitivity,mae_latent,mae_lasso,mae_ridge,true_h,true_pi,beta_var,h_gcta,mae_gcta,h_cover,pi_cover
0,0.785826,0.828413,0.734330,11.958570,3.267713,0.076733,0.150797,0.025402,0.151515,0.318182,0.725920,1.014471,1.032728,0.80,0.100,0.1,0.433485,4.229898,1,1
1,0.781825,0.826767,0.730461,4.963962,1.390744,0.074199,0.144028,0.025030,0.218750,0.284091,0.492906,0.667637,0.657780,0.80,0.100,0.1,0.491880,1.471430,1,1
2,0.789928,0.828357,0.746261,8.731348,2.324597,0.076745,0.143884,0.029282,0.181818,0.306818,0.631336,0.859025,0.868210,0.80,0.100,0.1,0.390950,1.589764,1,1
3,0.786165,0.831914,0.734533,13.833671,3.773741,0.063757,0.136126,0.017645,0.071429,0.295455,0.842101,1.125438,1.124868,0.80,0.100,0.1,0.416797,2.780782,1,1
4,0.787803,0.829373,0.742744,9.420783,2.543586,0.078986,0.150605,0.027933,0.029412,0.375000,0.626751,0.899535,0.895874,0.80,0.100,0.1,0.463655,1.794607,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.240832,0.307432,0.184613,0.585936,1.879982,0.010955,0.042728,0.000132,0.000000,1.000000,0.562440,0.637244,0.656467,0.25,0.005,0.1,0.101536,0.662705,1,1
5996,0.245624,0.311102,0.188273,0.160591,0.501665,0.011332,0.043627,0.000175,0.200000,1.000000,0.287781,0.338897,0.342997,0.25,0.005,0.1,0.147048,0.337225,1,1
5997,0.258520,0.325202,0.196579,0.192240,0.562163,0.009019,0.040227,0.000045,0.333333,0.500000,0.284216,0.335727,0.345284,0.25,0.005,0.1,0.149356,0.343391,1,1
5998,0.247079,0.331149,0.175694,0.631538,1.977193,0.013501,0.054969,0.000109,0.200000,1.000000,0.536385,0.626299,0.631932,0.25,0.005,0.1,0.095010,0.639185,1,1
